In [1]:
import numpy as np
# import math
import torch
# from torch.utils.data import Dataset
import uproot
# from glob import glob
# from tqdm import tqdm
# from bisect import bisect_right
import h5py
import torch.nn as nn
import torch.nn.functional as F
import torch_geometric.nn as PyG
from torch_geometric.transforms import Distance
from torch_geometric.data import Data


for k in range(100):
    print(k)
    file_name = "/store/hep/users/jhgoh/Delphes/20200530_1/noPU/QCDBkg/HT2000toInf/0000/QCDBkg_HT_2000_Inf-" + str(k+81) + ".root"
    RPV_raw = uproot.open(file_name)

    RPV = RPV_raw["Delphes"]

    a = np.asarray(RPV.array("Jet.PT"))
    b = np.asarray(RPV.array("Jet.Eta"))
    c = np.asarray(RPV.array("Jet.Phi"))
    d = np.asarray(RPV.array("Jet.T"))
    e = np.asarray(RPV.array("Jet.Mass"))


    pos = [[] for _ in range(len(a))]
    x_fea = [[] for _ in range(len(a))]

    for j in range(len(a)):
        for i in range(len(a[j])):
            x = np.cos(c[j][i])
            y = np.sin(c[j][i])
            z = np.sinh(b[j][i])
            pos_p = [x, y, z]
            x_fea_p = [a[j][i], d[j][i], e[j][i]]

            pos[j].append(pos_p)
            x_fea[j].append(x_fea_p)

    edge_t = [[] for _ in range(len(pos))]

    for i in range(len(pos)):
        x = torch.Tensor(pos[i])
#         edge_index = PyG.radius_graph(x, r = 100)
        edge_index = PyG.knn_graph(x, k = 2)
        edge_t[i].append(edge_index)

    pos_flat = [[] for _ in range(len(pos))]
    x_fea_flat = [[] for _ in range(len(pos))]
    edge_flat = [[] for _ in range(len(pos))]

    for i in range(len(pos)):
        pos_f = [y for x in pos[i] for y in x]
        x_fea_f = [y for x in x_fea[i] for y in x]

        pos_flat[i] = pos_f
        x_fea_flat[i] = x_fea_f


    for i in range(len(pos)):
        edge_f = [y for x in edge_t[i][0] for y in x]
        edge_flat[i] = edge_f
    save_f_name = 'QCD_3_' + str(k+79)
    f = h5py.File(save_f_name + '.hdf5', mode='w')
    dt = h5py.special_dtype(vlen=np.dtype('float64'))
    g = f.create_group("group")

    pos_data = g.create_group("pos")
    fea_data = g.create_group("fea")
    edge_data = g.create_group("edge")


    pos_data.create_dataset('pos', (len(a),), dtype=dt)
    pos_data['pos'][...] = pos_flat

    fea_data.create_dataset('fea', (len(a),), dtype=dt)
    fea_data['fea'][...] = x_fea_flat

    edge_data.create_dataset('edge', (len(a),), dtype=dt)
    edge_data['edge'][...] = edge_flat

    f.close()

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20


FileNotFoundError: [Errno 2] No such file or directory: '/store/hep/users/jhgoh/Delphes/20200530_1/noPU/QCDBkg/HT2000toInf/0000/QCDBkg_HT_2000_Inf-101.root'